# Notes
The command !pip install unsloth trl peft accelerate bitsandbytes installs a suite of Python libraries commonly used for fine-tuning and deploying large language models efficiently. unsloth provides fast and memory-efficient tools for fine-tuning models like Mistral and Phi using LoRA (Low-Rank Adaptation) with optimized performance. trl (Transformers Reinforcement Learning) includes tools like SFTTrainer for supervised fine-tuning of LLMs. peft (Parameter-Efficient Fine-Tuning) enables LoRA and other lightweight fine-tuning methods. accelerate simplifies multi-GPU and mixed-precision training setups. Finally, bitsandbytes allows models to be quantized to 8-bit or 4-bit, reducing memory usage significantly and enabling model training or inference on consumer GPUs like T4 or 3060. This combined setup is ideal for training powerful LLMs even on limited hardware.

In [ ]:
!pip install unsloth trl peft accelerate bitsandbytes #

alpaca_prompt

The alpaca_prompt is a multi-line string template that is used for training a model to generate SQL queries.

Instruction Section:

### Instruction:

This section is where you describe the task or the SQL query in a human-readable way. For example, the database context or the SQL task you want to generate.

{company_database}: A placeholder for the actual database schema or description (such as the tables and relationships involved in the SQL query).

Input Section:

### Input:

This is where the SQL prompt/question is given as input.

{prompt}: A placeholder for the SQL query prompt that you want the model to interpret and generate SQL for. This is typically a user query like "What is the total sales in the North region?"

Response Section:

### Response:

This is the expected output section, where the model is supposed to generate a response.

{sql}: The placeholder for the actual SQL query that should be generated by the model based on the prompt. For example, SELECT SUM(sales) FROM sales_table WHERE region = 'North'.

{explanation}: This is the explanation of how the SQL query was derived or why it solves the problem in the prompt.

<|endoftext|>:

This is a special token that signifies the end of the text in certain prompt designs. It helps in tokenizing and processing input and output efficiently, especially in sequence-based tasks like text generation.

# Salesforce/codegen-350M-mono
Salesforce/codegen-350M-mono is a specific model identifier within the CodeGen family developed by Salesforce.

350M: Refers to the model's size, meaning it has 350 million parameters. This is a mid-sized model in terms of the number of parameters, providing a balance between computational efficiency and performance.

mono: This refers to the monolingual version of the model. It is trained on a single language, which means it focuses on understanding and generating code in one specific language. The "mono" variant contrasts with a multilingual model that would support multiple languages.

 Model Purpose
Code Generation: The codegen-350M-mono model is fine-tuned for generating code, making it highly suitable for tasks such as text-to-SQL generation, code completion, code translation, and other coding-related applications.

The model can take in a prompt (natural language or some code) and generate code that solves a problem or completes a task based on the input.

**Important parameters for the tokenizer include:**

padding: Controls how the model handles sequences of varying lengths (i.e., sequences of text longer than the model's maximum sequence length).

truncation: This parameter ensures that text longer than the model's input length is truncated to fit within the maximum input length.

max_length: This specifies the maximum number of tokens allowed for the input sequence. For models like Salesforce/codegen-350M-mono, you would typically choose a max_length that corresponds to the model's capacity.

return_tensors: Defines the format of the output (e.g., pt for PyTorch tensors). This is required when passing the input to the model for inference.

add_special_tokens: Ensures that special tokens (like [CLS], [SEP], etc.) are added during tokenization.

**Model Parameters**
max_length: Refers to the maximum sequence length of tokens the model can process. For example, a model may have a max_length of 512 tokens, meaning any input sequence longer than 512 tokens would need to be truncated or split.

temperature: Affects the randomness of the model's output. Higher temperatures (e.g., 0.8 or 1.0) make the model's output more random, while lower temperatures (e.g., 0.2 or 0.3) make the model's output more deterministic.

top_p (nucleus sampling): Affects the diversity of the output by controlling the cumulative probability of the most likely next tokens. Only tokens within the top p cumulative probability are considered.

top_k: Restricts the sampling pool to the top k most likely tokens, limiting the possible outputs at each step.

do_sample: If set to True, the model generates text by sampling from the distribution of possible next tokens, adding randomness to the generated sequence.

use_cache: If set to True, the model will cache the past attention state for faster inference.


# Code


In [2]:
from datasets import Dataset                #This imports the Dataset class from Hugging Face's datasets library. It's used to convert data (like from a Pandas DataFrame or CSV) into a format that can be used directly with Hugging Face tools like transformers.
from transformers import AutoTokenizer
import json
import pandas as pd
splits = {'train': 'synthetic_text_to_sql_train.snappy.parquet', 'test': 'synthetic_text_to_sql_test.snappy.parquet'}
df = pd.read_parquet("hf://datasets/gretelai/synthetic_text_to_sql/" + splits["train"])
df.isnull().sum()
dataset = Dataset.from_pandas(df)
#These are paths to two dataset files in Parquet format (compressed with Snappy), often used for efficient data storage and access.
# Define Alpaca-style SQL prompt template
alpaca_prompt = """Below is an instruction that describes a task, paired with an input and expected output.
### Instruction:
Company database: {company_database}

### Input:
SQL Prompt: {prompt}

### Response:
SQL: {sql}
Explanation: {explanation}<|endoftext|>"""


# Format each example into a single text string
def format_prompt(example):
    return alpaca_prompt.format(
        company_database=example['sql_context'],
        prompt=example['sql_prompt'],
        sql=example['sql'],
        explanation=example['sql_explanation']
    )

# Format your dataset from pandas dataframe `df`
formatted_data = [format_prompt(row) for _, row in df.iterrows()]
dataset = Dataset.from_dict({"text": formatted_data})

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-350M-mono")

# Ensure pad token exists
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Done! Now pass this dataset to SFTTrainer like this:
# trainer = SFTTrainer(
#     model=model,
#     tokenizer=tokenizer,
#     train_dataset=dataset,
#     dataset_text_field="text",
#     ...
# )

# Optional: preview first row
print(dataset[0]['text'])
dataset = dataset.select(range(200))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Below is an instruction that describes a task, paired with an input and expected output.
### Instruction:
Company database: CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith', 'South'); CREATE TABLE timber_sales (sales_id INT, salesperson_id INT, volume REAL, sale_date DATE); INSERT INTO timber_sales (sales_id, salesperson_id, volume, sale_date) VALUES (1, 1, 120, '2021-01-01'), (2, 1, 150, '2021-02-01'), (3, 2, 180, '2021-01-01');

### Input:
SQL Prompt: What is the total volume of timber sold by each salesperson, sorted by salesperson?

### Response:
SQL: SELECT salesperson_id, name, SUM(volume) as total_volume FROM timber_sales JOIN salesperson ON timber_sales.salesperson_id = salesperson.salesperson_id GROUP BY salesperson_id, name ORDER BY total_volume DESC;
Explanation: Joins timber_sales and salesperson tables, groups sales by salesperson, calculates total

Why is this important?
Padding Tokens: Padding ensures that all sequences in a batch are the same length, which is necessary for efficient processing in parallel on GPUs.

eos_token as Padding: If the tokenizer doesn’t define a padding token, using the eos_token as padding works because it allows the model to treat padding as part of the sequence, rather than using a separate, dedicated token. This is especially useful for sequence generation tasks like code generation or text generation, where the eos_token marks the end of the sequence and could also serve as padding.

In [1]:
# For GPU check
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

CUDA available: True
GPU: Tesla T4


In [3]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/Phi-3-mini-4k-instruct-bnb-4bit"

max_seq_length = 2048  # Choose sequence length
dtype = None  # Auto detection

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

/tmp/ipython-input-3-4282502808.py:1: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.12: Fast Mistral patching. Transformers: 4.53.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
# from datasets import Dataset

# def format_prompt(example):
#     return f"### Input: {example['input']}\n### Output: {json.dumps(example['output'])}<|endoftext|>"

# formatted_data = [format_prompt(item) for item in file]
# dataset = Dataset.from_dict({"text": formatted_data})

Parameters:
r=64 (LoRA Rank):

LoRA rank (r) refers to the size of the low-rank matrix used for adaptation. The higher the rank, the more capacity the model has for learning the low-rank transformations, but at the cost of more memory and computation. A rank of 64 suggests that the low-rank matrices will have 64-dimensional projections.

target_modules:

This specifies which parts of the model will be fine-tuned using the LoRA adapters. Each item refers to a specific attention or projection layer in the transformer architecture.

"q_proj", "k_proj", "v_proj", "o_proj": These are the query, key, value, and output projections used in the attention layers of the transformer model.

"gate_proj", "up_proj", "down_proj": These are other parts of the model (such as those used in feed-forward layers, gate functions, or other internal layers) that will be adapted.

lora_alpha=128:

The scaling factor for the LoRA layers. This helps scale the low-rank updates in the model. A typical value is 2x the rank (so lora_alpha = 2 * r).

A larger value here means that the low-rank adaptation will have a higher influence on the model.

lora_dropout=0:

This refers to dropout used in the LoRA layers. Dropout helps prevent overfitting during training by randomly dropping some connections. Here, it's set to 0, meaning no dropout is used.

bias="none":

Specifies that the LoRA adapters will not introduce bias terms. LoRA typically operates without bias for efficiency.

use_gradient_checkpointing="unsloth":

Gradient checkpointing is used to reduce memory consumption during training by saving only certain layers of gradients and recomputing others. The "unsloth" here refers to a custom, optimized version provided by the Unsloth framework for better performance.

random_state=3407:

A fixed random state value to ensure reproducibility of the training process. This ensures that if you run the training multiple times, the results will be consistent.

use_rslora=False:

This parameter controls whether to use Rank Stabilized LoRA (rSLORA), a variant that stabilizes the rank of the low-rank approximations during fine-tuning. It's set to False, meaning the standard LoRA is used.

loftq_config=None:

LoftQ is a specific configuration that can be used with LoRA to further optimize training with quantized weights. Since it's set to None, it means LoftQ is not used in this case.

In [4]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=64,  # LoRA rank - higher = more capacity, more memory
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=128,  # LoRA scaling factor (usually 2x rank)
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",     # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # Unsloth's optimized version
    random_state=3407,
    use_rslora=False,  # Rank stabilized LoRA
    loftq_config=None, # LoftQ
)

Unsloth 2025.6.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Parameters and Their Explanation:
model:

The model that has been fine-tuned or is ready for fine-tuning. It's passed to the SFTTrainer, which will be used for training.

tokenizer:

The tokenizer that corresponds to the model. It is used for encoding and decoding inputs and outputs during training and evaluation.

train_dataset=dataset:

The training dataset to be used. This dataset will be used to train the model on specific tasks (e.g., SQL generation).

dataset_text_field="text":

This indicates the field in the dataset that contains the input text. In this case, the dataset field is named text, which is where the formatted data (SQL queries, explanations, etc.) will be stored.

max_seq_length=max_seq_length:

The maximum sequence length that the model will process. Any input longer than this will be truncated.

dataset_num_proc=2:

Specifies the number of CPU processes used to process the dataset during training. Using more processes can speed up data processing, especially for large datasets.

TrainingArguments (Key Hyperparameters):
per_device_train_batch_size=2:

Specifies the batch size per device. Since it's set to 2, this means that for each GPU (or CPU), 2 samples will be processed in parallel during each step.

gradient_accumulation_steps=4:

Gradient Accumulation allows for effective batch sizes larger than the device's physical memory limit. In this case, with 4 gradient accumulation steps, the model will accumulate gradients over 4 steps before updating the weights, which gives an effective batch size of 8 (2 per device * 4).

warmup_steps=10:

Warmup steps indicate how many steps the learning rate will gradually increase from 0 to the initial value during training. It helps in stabilizing the model's training early on.

num_train_epochs=3:

The number of epochs the model will be trained for. In this case, the model will train for 3 full passes through the dataset.

learning_rate=2e-4:

The learning rate is set to 0.0002. It controls the size of the steps the optimizer takes when updating the model's weights during training.

fp16=not torch.cuda.is_bf16_supported() and bf16=torch.cuda.is_bf16_supported():

These flags enable mixed precision training depending on whether bfloat16 (bf16) is supported on the device. This helps save memory and speeds up training.

logging_steps=25:

The logging frequency is set to 25 steps. This means that the training process will log information such as loss, learning rate, etc., every 25 steps.

optim="adamw_8bit":

Specifies the optimizer to use during training. Here, the AdamW optimizer with 8-bit precision is used. 8-bit optimizers reduce the memory footprint, allowing for faster training with larger models.

weight_decay=0.01:

Weight decay helps regularize the model by penalizing large weights, which reduces overfitting.

lr_scheduler_type="linear":

The learning rate scheduler determines how the learning rate changes during training. Here, it’s set to "linear," meaning the learning rate will decrease linearly from its initial value to zero.

seed=3407:

The random seed is set to 3407 to ensure reproducibility of the results. Using the same seed, you will get the same random number generation, making the training process reproducible.

output_dir="outputs":

The directory where the trained model will be saved at the end of training.

save_strategy="epoch":

This defines how often the model will be saved. In this case, it will be saved at the end of every epoch.

save_total_limit=2:

The model will keep only the last 2 saved models. Older models will be automatically deleted to save disk space.

dataloader_pin_memory=False:

Disables pinning memory during data loading. Pinning memory can sometimes improve performance, but it's not necessary in all cases.

report_to="none":

This disables reporting metrics and training logs to platforms like Weights & Biases. This is useful if you do not need such external integrations.

In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Training arguments optimized for Unsloth
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,  # Effective batch size = 8
        warmup_steps=10,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=25,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False,
        report_to="none", # Disable Weights & Biases logging
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/200 [00:00<?, ? examples/s]

In [6]:
# Train the model
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 200 | Num Epochs = 3 | Total steps = 75
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 119,537,664 of 2,128,677,888 (5.62% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
25,0.552800
50,0.357000
75,0.282600


In [7]:
# If using PEFT (LoRA adapter)
from peft import PeftModel

# Save and push
model.save_pretrained("mymodel")
tokenizer.save_pretrained("mymodel")

('mymodel/tokenizer_config.json',
 'mymodel/special_tokens_map.json',
 'mymodel/chat_template.jinja',
 'mymodel/tokenizer.model',
 'mymodel/added_tokens.json',
 'mymodel/tokenizer.json')

# Results

In [9]:
import torch

# Enable Unsloth 2x faster inference
from unsloth import FastLanguageModel
FastLanguageModel.for_inference(model)

# ---- 1. Build prompt just like during training ----
def build_prompt(company_db, sql_prompt):
    return f"""Below is an instruction that describes a task, paired with an input and expected output.
### Instruction:
Company database: {company_db}

### Input:
SQL Prompt: {sql_prompt}

### Response:"""

# ---- 2. Define your question ----
company_database = "CREATE TABLE employees (id INT, name TEXT, salary REAL, department TEXT);"
question = "What is the average salary per department?"

# ---- 3. Format the full prompt ----
prompt = build_prompt(company_database, question)

# ---- 4. Tokenize input prompt ----
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# ---- 5. Generate response ----
with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=256,
        temperature=0.7,
        do_sample=True,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id,
    )

# ---- 6. Decode and clean output ----
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
response_only = decoded_output.split("### Response:")[-1].split("<|endoftext|>")[0].strip()

# ---- 7. Print the final result ----
print("=== Model Response ===\n", response_only)


=== Model Response ===
 SQL: SELECT department, AVG(salary) FROM employees GROUP BY department;
Explanation: The SQL query calculates the average salary for each department by grouping the records by the department column and applying the AVG function to the salary column.


In [10]:
def build_prompt(db_schema, sql_prompt):
    return f"""Below is an instruction that describes a task, paired with an input and expected output.
### Instruction:
Company database: {db_schema}

### Input:
SQL Prompt: {sql_prompt}

### Response:"""

test_cases = [
    ("CREATE TABLE employees (id INT, name TEXT, salary REAL);", "What is the total number of employees?"),
    ("CREATE TABLE employees (id INT, name TEXT, salary REAL);", "Get the names of employees who earn more than 50000."),
    ("CREATE TABLE employees (id INT, name TEXT, salary REAL, department TEXT);", "What is the average salary per department?"),
    ("CREATE TABLE employees (id INT, name TEXT, salary REAL);", "Who is the highest-paid employee?"),
    ("CREATE TABLE employees (id INT, name TEXT, department TEXT);", "List departments that have more than 5 employees."),
]

for db_schema, prompt in test_cases:
    full_prompt = build_prompt(db_schema, prompt)
    inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=256,
        temperature=0.7,
        do_sample=True,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id,
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"❓ Prompt: {prompt}\n📄 Response:\n{response.split('### Response:')[-1].split('<|endoftext|>')[0].strip()}\n{'-'*80}")


❓ Prompt: What is the total number of employees?
📄 Response:
SQL: SELECT COUNT(*) FROM employees;
Explanation: This SQL query calculates the total number of employees by using the COUNT function to count the number of rows in the employees table.
--------------------------------------------------------------------------------
❓ Prompt: Get the names of employees who earn more than 50000.
📄 Response:
SQL: SELECT name FROM employees WHERE salary > 50000;
Explanation: This query retrieves the names of employees who earn more than 50000 by selecting the name column from the employees table where the salary is greater than 50000.
--------------------------------------------------------------------------------
❓ Prompt: What is the average salary per department?
📄 Response:
SQL: SELECT department, AVG(salary) FROM employees GROUP BY department;
Explanation: This SQL query calculates the average salary per department. It uses the AVG() function to calculate the average value of the salary col

In [ ]:
# Test the fine-tuned model
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# Test prompt
messages = [
    {"role": "user", "content": "Extract the product information:\n<div class='product'><h2>iPad Air</h2><span class='price'>$1344</span><span class='category'>audio</span><span class='brand'>Dell</span></div>"},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

# Generate response
outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=256,
    use_cache=True,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
)

# Decode and print
response = tokenizer.batch_decode(outputs)[0]
print(response)

In [ ]:
model.save_pretrained_gguf("gguf_model", tokenizer, quantization_method="q4_k_m")

In [ ]:
from google.colab import files
import os

gguf_files = [f for f in os.listdir("gguf_model") if f.endswith(".gguf")]
if gguf_files:
    gguf_file = os.path.join("gguf_model", gguf_files[0])
    print(f"Downloading: {gguf_file}")
    files.download(gguf_file)

unsloth: A library that provides fast and efficient models, including features for working with PEFT (Parameter Efficient Fine-Tuning) adapters, and the ability to load models optimized for low-precision computation.

torch: PyTorch is a deep learning framework, and it’s imported to handle tensor operations and work with GPU acceleration.

max_seq_length: Defines the maximum sequence length (2048 tokens). This parameter ensures that the input sequences don’t exceed this length when being tokenized.

dtype: Specifies the data type used for the model’s computations (e.g., float16, float32). Setting it to None means it will auto-detect the most appropriate type based on the hardware.

load_in_4bit=True: This option loads the model in 4-bit precision, optimizing memory usage and improving performance, especially useful for large models that may otherwise be too memory-intensive for your GPU.